In [1]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import shapely
from shapely.geometry import mapping
# from shapely.geometry.polygon import Polygon
# from shapely.geometry.multipolygon import MultiPolygon
from explode import explode
from coord import coord
import matplotlib.pyplot as plt

In [2]:
#load data layers
f = '/Users/Zack/0_seawarden/targets_sar_greece/'
#n = '2019_g25_100m_10-20_6m_02-01'
n = '2019_g25_100m_10-20_1yr_02-01'
#n = '2018_g25_100m_10-20_6m_02-02'
#n = '2018_g25_100m_10-20_1yr_02-02'

pts = gpd.read_file(f + n + '.geojson')

f2 = '/Users/Zack/0_seawarden/'
search_area = gpd.read_file(f2 + 'search_areas/4_search_area/search_area_100m_greece.shp')
exclusion_zones = gpd.read_file(f2 + 'search_areas/7_exclusions/GRC_exclusions_2020-01-07.geojson')
farm_sites = gpd.read_file(f2 + 'greece/blue_bridge/farm_extent_216.shp')

#re-project layers
pts = pts.to_crs({'init': 'epsg:2100'})
search_area = search_area.to_crs({'init': 'epsg:2100'})
exclusion_zones = exclusion_zones.to_crs({'init': 'epsg:2100'})

print('detections', len(pts), pts.crs)
print('search area', len(search_area), search_area.crs)
print('exclusion zones', len(exclusion_zones), exclusion_zones.crs)

detections 10038 {'init': 'epsg:2100'}
search area 1 {'init': 'epsg:2100'}
exclusion zones 15 {'init': 'epsg:2100'}


In [3]:
# plt.rcParams['figure.figsize'] = (20, 10)
# ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
# pts.plot(markersize=5, facecolor='red', ax=ax)
# exclusion_zones.plot(ax=ax)

In [4]:
%%time 
#clip points to search area
clip = sjoin(pts, search_area, how='inner', op='within')
print('detections within search area:', len(clip))

detections within search area: 4381
Wall time: 2.43 s


In [5]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#clip.plot(markersize=5, facecolor = 'red', ax=ax)

In [6]:
%%time 
#combine all exclusion zones
exclusion_zones['Dissolve'] = 0
exclusion_zones_dis = exclusion_zones.dissolve(by='Dissolve')

Wall time: 12.2 ms


In [7]:
%%time 
#exclude points in exclusion areas
mask = ~clip.within(exclusion_zones_dis.loc[0, 'geometry'])
clip2 = clip.loc[mask]
print('detections outside of exclusion zones:', len(clip2))

detections outside of exclusion zones: 3410
Wall time: 232 ms


In [8]:
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#exclusion_zones_dis.plot(facecolor = 'blue', ax=ax)
#clip2.plot(markersize=5, facecolor = 'red', ax=ax)

In [9]:
%%time 
#buffer points, dissolve buffers to aggregate points near each other
buffer = gpd.GeoDataFrame(geometry = clip2.buffer(10))
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')

Wall time: 328 ms


In [10]:
%%time 
#explode polygon and generate centroids
buffer_exploded = explode(buffer_dis)      
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid)
print('detections after aggregation:', len(centroids))

detections after aggregation: 3393
Wall time: 1.31 s


In [11]:
%%time 
#buffer centroids and make square polygons
centroid_buffer = gpd.GeoDataFrame(geometry = centroids.buffer(500))
envelope = gpd.GeoDataFrame(geometry = centroid_buffer.envelope)

Wall time: 194 ms


In [12]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroid_buffer.plot(markersize=5, facecolor = 'black', ax=ax)
#envelope.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [13]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax = centroids.loc[[0], 'geometry'].plot(markersize=10, facecolor = 'black')
#centroid_buffer.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)
#envelope.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [14]:
#set crs
envelope.crs = {'init' :'epsg:2100'}
envelope['geometry'] = envelope['geometry'].to_crs(epsg=4326)
print(envelope.crs)
envelope.head()

{'init': 'epsg:2100'}


,geometry
0,"POLYGON ((24.10679575932637 34.78183294138574,..."
1,"POLYGON ((24.04270862259479 34.83592457621792,..."
2,"POLYGON ((24.11350216291329 34.84261271388799,..."
3,"POLYGON ((24.0377078200892 34.84627264441792, ..."
4,"POLYGON ((24.03327759458833 34.84953410151513,..."


In [15]:
#extract lat/long for each square polygon (envelope)
coord_list = []
for i in range(len(envelope)):
    coords = mapping(envelope.geometry[i])['coordinates']
    coord_list.append(coords)

In [16]:
# #combine x/y point groups
coord_all = []
for i in range(len(coord_list)):
    coord_group = coord(coord_list[i]) #function to extract and format x/y points
    coord_group['id']=i
    coord_all.append(coord_group)
    
targets = pd.concat(coord_all)
print('total coordinates (5 per detection):', len(targets))
targets.head()

total coordinates (5 per detection): 16965


,x,y,id
0,24.106795759326367,34.78183294138574,0
1,24.117725485820888,34.78182286292244,0
2,24.117738298665376,34.79084058077296,0
3,24.106807382622108,34.79085066260232,0
4,24.106795759326367,34.78183294138574,0


In [17]:
#accuracy check
centroids_test = centroids.copy()
centroids_test.crs = {'init' :'epsg:2100'}
centroids_test['geometry'] = centroids_test['geometry'].to_crs(epsg=4326)

matches = centroids_test.intersects(farm_sites.unary_union)
count = centroids_test.loc[matches]

detections_n = len(centroids_test)
farm_n = len(farm_sites)
matches_n = len(count)

print('total detections:', len(pts))
print('detections in search area:', detections_n)
print('farm sites:', farm_n)
print('matches:', matches_n)
print('positive matches', round(matches_n / farm_n, 3))
print('overall accuracy', round(matches_n / detections_n, 3))

#plt.rcParams['figure.figsize'] = (20, 10)
#ax=farm_sites.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroids_test.plot(markersize=1, facecolor = 'red', ax=ax)

total detections: 10038
detections in search area: 3393
farm sites: 216
matches: 293
positive matches 1.356
overall accuracy 0.086


In [18]:
pts.to_file(f2 + n + '.shp')
targets.to_csv(f2 + n + '.csv', index = None, header=True)

#centroids_test.crs = {'init' :'epsg:4326'}
#centroids_test.to_file(f2 + n + '_v2.shp')
#centroids_test.to_file(f2 + n + '_v2.geojson', driver='GeoJSON')